<a href="https://colab.research.google.com/github/SerasLain/NLP-2019/blob/master/%5CHW_2%5CNLP_HW2_Sidorova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP-2019, HW2
### Выполнила Анастасия Сидорова БКЛ162

In [4]:
!pip install textdistance

  Stored in directory: /root/.cache/pip/wheels/7b/b5/d2/cbccb7aab168929e7871577bf798c677ca820603dd1210bccb
Successfully built textdistance


In [0]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
punct = set(punctuation)
from sklearn.metrics import classification_report
from difflib import get_close_matches
import textdistance

In [0]:
def get_closest_match_with_metric(text, lookup, metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    return similarities.most_common(1)[0]

In [0]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [0]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()
corpus = open('corpus_ng.txt', encoding='utf-8').read().split(' ')
WORDS = Counter()
WORDS.update(corpus)

In [22]:
# Проверяем, что все нормально на слова разбилось
print(WORDS)

Counter({'в': 3173, 'и': 2775, 'на': 1466, 'не': 1088, '': 912, 'что': 844, 'с': 784, 'по': 676, 'а': 434, 'но': 426, 'как': 419, 'к': 414, 'для': 372, 'из': 363, 'это': 325, 'за': 306, 'о': 288, 'от': 267, 'его': 259, 'то': 217, 'он': 214, 'все': 206, 'до': 205, 'так': 198, 'же': 196, 'у': 195, 'более': 174, 'россии': 164, 'уже': 161, 'или': 158, 'было': 157, 'года': 156, 'при': 154, 'их': 152, 'бы': 150, 'только': 149, 'я': 136, 'году': 136, 'которые': 135, 'если': 135, 'был': 132, 'этом': 132, 'также': 127, 'есть': 125, 'еще': 123, 'будет': 123, 'страны': 121, 'время': 118, 'мы': 117, 'они': 117, 'после': 116, 'ее': 112, 'том': 111, 'чтобы': 110, 'можно': 107, 'того': 104, 'может': 100, 'даже': 98, 'во': 98, 'лет': 97, 'были': 94, 'сша': 94, 'тем': 92, 'когда': 90, 'чем': 90, 'была': 89, 'очень': 85, 'который': 84, 'где': 83, 'них': 82, 'под': 76, 'этого': 75, 'она': 75, 'со': 73, 'этот': 72, 'тыс': 72, 'ли': 69, 'всего': 69, 'однако': 67, 'раз': 66, 'м': 66, 'развития': 66, 'ни': 6

In [0]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [0]:
def get_closest_match_vec(text, X, vec, TOPN=3):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]
    
    return [id2word[top] for top in topn]

In [0]:
# Находит ближайшее по векторам, а потом ищет ближайшее по метрике из предложенных
def get_closest_hybrid_match(text, X, vec, metric=textdistance.levenshtein):
    candidates = set(get_closest_match_vec(text, X, vec))
    closest = get_closest_match_with_metric(text, candidates)
    return closest[0]

In [0]:
def align_words(true, bad):
    tokens_true = true.lower().split()
    tokens_bad = bad.lower().split()
    tokens_true = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_true if (set(token)-punct)]
    tokens_bad = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_bad if (set(token)-punct)]
    corrected = []
    for token in tokens_bad:
      if token in vocab:
        corrected.append(token)
      else:
        corrected.append(get_closest_hybrid_match(token, X, vec))
    return list(zip(tokens_true, tokens_bad, corrected))

In [0]:
def eval():
  # Ошибок, если ничего не делать
  # Ошибок после исправления
  y_mistakes = 0
  y_corr_mistakes = 0
  total = 0
  miscorrected = []
  for s in range(len(bad)):
    trios = align_words(true[s], bad[s])
    for trio in trios:
      total += 1
      if trio[0] != trio[1]:
        y_mistakes += 1
      if trio[0] != trio[2]:
        miscorrected.append(trio)
        y_corr_mistakes += 1
  return y_mistakes, y_corr_mistakes, total, miscorrected

In [37]:
y_mistakes, y_corr_mistakes, total, miscorrected = eval()
print('Если ничего не делать\t', y_mistakes/total)
print('После исправления\t', y_corr_mistakes/total)

Если ничего не делать	 0.13034358769476628
После исправления	 0.3421893727526968


In [38]:
# Статистика не очень, посмотрим на ошибки
print(miscorrected[:20])

[('симпатичнейшее', 'симпатичнейшое', 'чистейшими'), ('шпионское', 'шпионское', 'шапошников'), ('гламурный', 'гламурный', 'суммарный'), ('фотоаппарат', 'фотоаппарат', 'филантропа'), ('бонда', 'бонда', 'банд'), ('миниатюрная', 'миниатюрная', 'миниатюре'), ('камеры', 'камеры', 'камерные'), ('superheadz', 'superheadz', 'european'), ('clap', 'clap', 'capital'), ('camera', 'camera', 'caterham'), ('апофеозом', 'опофеозом', 'фанерозое'), ('услышанная', 'услышанная', 'услышана'), ('поясним', 'пояним', 'понимая'), ('получатся', 'полчатся', 'получается'), ('язычки', 'язычки', 'языки'), ('очень', 'оччччень', 'очерчен'), ('милые', 'милые', 'малые'), ('насчет', 'нащщот', 'ощущают'), ('чавеса', 'чавеса', 'часа'), ('учеником', 'учеником', 'учениками')]


Ошибки есть такие:
- незнакомые корпусу слова (сбалансировать корпус какими-нибудь более адекватными текстами)
- латиница и цифры (убрать цифры и латиницу из рассмотрения, не исправлять их или для латиницы добавить еще и корпуса на английском)
- незнакомые корпусу формы (либо брать в руки всякое pymorphy и сначала каждое незнакомое слово стемить и искать в словаре, либо расширять корпус)
- слова со схожим написанием, но разным смыслом, ошибки типа правописания не/ни (исследовать контекст?)
- слово ошибочно написано слитно - оно сами и еще несколько слов после него становятся ошибочными

In [0]:
# Перестанем исправлять цифры

def align_words(true, bad):
    tokens_true = true.lower().split()
    tokens_bad = bad.lower().split()
    tokens_true = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_true if (set(token)-punct)]
    tokens_bad = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_bad if (set(token)-punct)]
    corrected = []
    for token in tokens_bad:
      if token.isdigit():
        corrected.append(token)
      if token in vocab:
        corrected.append(token)
      else:
        corrected.append(get_closest_hybrid_match(token, X, vec))
    return list(zip(tokens_true, tokens_bad, corrected))


In [0]:
# Увеличим корпус: у меня под рукой есть роман Чернышевского "Что делать?" и я пущу его в ход (хотя по-хорошему, конечно, надо  поймать граммар-наци и вытрясти из него его вк-переписки, чтобы речь была максимально живой и современной)
chern = [token.lower().strip() for token in open('Что делать.txt', encoding='utf8').read().split(' ')]
chern = [token.strip(punctuation) for token in chern]
WORDS.update(chern)

In [54]:
# Проверим, помогло ли
y_mistakes, y_corr_mistakes, total, miscorrected = eval()
print('Если ничего не делать\t', y_mistakes/total)
print('После исправления\t', y_corr_mistakes/total)

Если ничего не делать	 0.13034358769476628
После исправления	 0.25339592489013185


Немного помогло, но явно недостаточно:
- Все еще правильные формы местами исправляются на неправильные (видимо, без pymystem тут никак)
- Некоторые формы исправляются неверно (возможно, имеет смысл брать побольше кандидатов, не три, а десять, например)
- Нужен нормальный словарь, не очень понятно, как это корпусом покрыть


In [55]:
print(miscorrected)

[('симпатичнейшее', 'симпатичнейшое', 'пластичнейшими'), ('шпионское', 'шпионское', 'шипенко'), ('гламурный', 'гламурный', 'гуманный'), ('фотоаппарат', 'фотоаппарат', 'филантропа'), ('бонда', 'бонда', 'банд'), ('superheadz', 'superheadz', 'deutsche'), ('clap', 'clap', 'culpa'), ('camera', 'camera', 'caterham'), ('услышанная', 'услышанная', 'услышана'), ('получатся', 'полчатся', 'постучался'), ('язычки', 'язычки', 'языки'), ('очень', 'оччччень', 'чечни'), ('милые', 'милые', 'милы'), ('насчет', 'нащщот', 'ощущать'), ('чавеса', 'чавеса', 'часа'), ('практическая', 'практическая', 'практически'), ('навыков', 'навыков', 'навык'), ('попавшим', 'попавшим', 'попавшие'), ('аварийно-спасательных', 'аварийно-спасательных', 'спасательных'), ('дорожно-транспортных', 'дорожно-транспортных', 'дорожно-транспортные'), ('выброшенные', 'выброшенные', 'брошенные'), ('в', 'вобщем', 'общем'), ('общем', 'как', 'как'), ('как', 'вы', 'вы'), ('вы', 'знаете', 'знаете'), ('знаете', 'из', 'из'), ('из', 'моего', 'мо